# 가위바위보 분류기

**trainset : 양재캠퍼스의 데이터(주먹 : 1432, 가위 : 1427, 보 : 1427개) + 강남캠퍼스 데이터(주먹:203, 가위: 203, 보: 203)**


**testset : 강남캠퍼스의 데이터(주먹 : 599, 가위: 599, 보: 599)**

trainset과 testest은 다른사람의 데이터를 사용하고 있습니다.

### 1. 데이터 가져오기 및 resize

In [1]:
from PIL import Image
import os, glob

print("PIL 라이브러리 import 완료!")

PIL 라이브러리 import 완료!


In [2]:
train_rsp_dict = {'rock' : 0, 'scissor' : 0, 'paper' : 0} #train 데이터의 바위, 가위, 보 갯수를 세어주는 dictionary
test_rsp_dict = {'rock' : 0, 'scissor' : 0, 'paper' : 0} #train 데이터의 바위, 가위, 보 갯수를 세어주는 dictionary

In [3]:
#train 데이터와 test데이터의 경로를 가진 리스트
train_file_path = ["/aiffel/aiffel/exploration/exploration1/rock", "/aiffel/aiffel/exploration/exploration1/scissor", "/aiffel/aiffel/exploration/exploration1/paper"]
test_file_path = ["/aiffel/aiffel/exploration/exploration1/rocks_complete", "/aiffel/aiffel/exploration/exploration1/scissors_complete", "/aiffel/aiffel/exploration/exploration1/papers_complete"]

In [4]:
import os

def train_resize_images(img_path):
    
    images = glob.glob(img_path + "/*.jpg")
    
    print(len(images), " images to be resized.")
    
    if 'rock' in img_path.split('/'):
        train_rsp_dict['rock'] = len(images)
    elif 'scissor' in img_path.split('/'):
        train_rsp_dict['scissor'] = len(images)
    else:
        train_rsp_dict['paper'] = len(images)
    # 파일마다 모두 28x28 사이즈로 바꾸어 저장
    
    target_size = (28, 28)
    for img in images:
        old_img = Image.open(img)
        new_img = old_img.resize(target_size, Image.ANTIALIAS)
        new_img.save(img, "JPEG")
    
    print(len(images), " images resized.")

In [5]:
for f in train_file_path:
    train_resize_images(f)

1641  images to be resized.
1641  images resized.
1636  images to be resized.
1636  images resized.
1636  images to be resized.
1636  images resized.


In [6]:
train_rsp_dict

{'rock': 1641, 'scissor': 1636, 'paper': 1636}

In [7]:
def test_resize_images(img_path):
    
    images = glob.glob(img_path + "/*.jpg")
    
    print(len(images), " images to be resized.")
    
    #주먹 가위 보가 각각 몇개씩 들어있는지 파악하도록 딕셔너리에 저장
    if 'rocks_complete' in img_path.split('/'):
        test_rsp_dict['rock'] = len(images)
    elif 'scissors_complete' in img_path.split('/'):
        test_rsp_dict['scissor'] = len(images)
    else:
        test_rsp_dict['paper'] = len(images)
    
    # 파일마다 모두 28x28 사이즈로 바꾸어 저장
    
    target_size = (28, 28)
    for img in images:
        old_img = Image.open(img)
        new_img = old_img.resize(target_size, Image.ANTIALIAS)
        new_img.save(img, "JPEG")
    
    print(len(images), " images resized.")

In [8]:
for f in test_file_path:
    test_resize_images(f)

599  images to be resized.
599  images resized.
599  images to be resized.
599  images resized.
599  images to be resized.
599  images resized.


### 2. 이미지를 train 과 test 행렬에 복사

In [9]:
import numpy as np

image_dir_path = "/aiffel/aiffel/exploration/exploration1/"
number_of_train_data = train_rsp_dict['rock'] + train_rsp_dict['scissor'] + train_rsp_dict['paper']


def load_train_data(img_path, number_of_train_data):  # 가위바위보 이미지 개수 총합에 주의하세요.
    # 가위 : 0, 바위 : 1, 보 : 2
    img_size=28
    color=3
    #train, test 각각의  이미지 데이터와 라벨(가위 : 0, 바위 : 1, 보 : 2) 데이터를 담을 행렬(matrix) 영역을 생성합니다.
    train_imgs = np.zeros(number_of_train_data*img_size*img_size*color,dtype=np.int32).reshape(number_of_train_data,img_size,img_size,color)
    train_labels = np.zeros(number_of_train_data, dtype=np.int32)
    
    train_idx = 0

    for file in glob.iglob(img_path+'/scissor/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        train_imgs[train_idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        train_labels[train_idx]=0   # 가위 : 0
        train_idx = train_idx+1


    for file in glob.iglob(img_path+'/rock/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        train_imgs[train_idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        train_labels[train_idx]=1   # 바위 : 1
        train_idx = train_idx+1

    for file in glob.iglob(img_path+'/paper/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        train_imgs[train_idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        train_labels[train_idx]=2   # 보 : 2
        train_idx = train_idx+1
     
    print("학습데이터(x_train)의 이미지 개수는", number_of_train_data,"입니다.")
    return train_imgs, train_labels

x_train, y_train = load_train_data(image_dir_path, number_of_train_data)
x_train_norm = x_train/255.0   # 입력은 0~1 사이의 값으로 정규화

print("x_train shape: {}".format(x_train.shape))
print("y_train shape: {}".format(y_train.shape))


학습데이터(x_train)의 이미지 개수는 4913 입니다.
x_train shape: (4913, 28, 28, 3)
y_train shape: (4913,)


In [10]:
import numpy as np

image_dir_path = "/aiffel/aiffel/exploration/exploration1/"
number_of_test_data = test_rsp_dict['rock'] + test_rsp_dict['scissor'] + test_rsp_dict['paper']

def test_load_data(img_path, number_of_test_data):  # 가위바위보 이미지 개수 총합에 주의하세요.
    # 가위 : 0, 바위 : 1, 보 : 2
    img_size=28
    color=3
    #train, test 각각의  이미지 데이터와 라벨(가위 : 0, 바위 : 1, 보 : 2) 데이터를 담을 행렬(matrix) 영역을 생성합니다.
    test_imgs = np.zeros(number_of_test_data*img_size*img_size*color,dtype=np.int32).reshape(number_of_test_data,img_size,img_size,color)
    test_labels = np.zeros(number_of_test_data,dtype=np.int32)

    test_idx = 0

    for file in glob.iglob(img_path+'/scissors_complete/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)      
        test_imgs[test_idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        test_labels[test_idx]=0    #   가위 : 0
        test_idx = test_idx+1

    for file in glob.iglob(img_path+'/rocks_complete/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        test_imgs[test_idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        test_labels[test_idx]= 1  # 바위 : 1
        test_idx = test_idx+1
    
    for file in glob.iglob(img_path+'/papers_complete/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        test_imgs[test_idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        test_labels[test_idx]=2   # 보 : 2
        test_idx = test_idx+1
        
    print("테스트데이터(x_test)의 이미지 개수는", number_of_test_data,"입니다.")
    return test_imgs, test_labels

x_test, y_test = test_load_data(image_dir_path, number_of_test_data)

# 입력은 0~1 사이의 값으로 정규화
x_test_norm = x_test/255.0

print("x_test shape: {}".format(x_test.shape))
print("y_test shape: {}".format(y_test.shape))

테스트데이터(x_test)의 이미지 개수는 1797 입니다.
x_test shape: (1797, 28, 28, 3)
y_test shape: (1797,)


### 3. 모델 생성 및 정확도 확인

In [11]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt

In [22]:
n_channel_1=32
n_channel_2=64
n_channel_3=128
#n_dense_1= 32
#n_dense_2 = 128
n_train_epoch=35

model=keras.models.Sequential()

model.add(keras.layers.Conv2D(n_channel_1, (3,3), activation='relu',input_shape=(28,28,3)))
model.add(keras.layers.MaxPool2D(2,2))
model.add(keras.layers.Dropout(rate = 0.5)) #Dropout을 통한 정규화
model.add(keras.layers.Conv2D(n_channel_2, (3,3), activation='relu'))
model.add(keras.layers.MaxPooling2D((2,2)))
model.add(keras.layers.Dropout(rate = 0.5)) #Dropout을 통한 정규화
model.add(keras.layers.Conv2D(n_channel_3, (3,3), activation='relu'))
#model.add(keras.layers.MaxPooling2D((2,2)))
#model.add(keras.layers.Flatten())
#model.add(keras.layers.Dense(32, activation='relu'))
model.add(keras.layers.GlobalAveragePooling2D()) #Maxpooling 대신 GAP를 FC Layer를 사용했을 때보다 사용하여 파라미터도 줄인다.
model.add(keras.layers.Dense(3, activation='softmax'))

model.summary()
model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy', #label이 정수값이므로 loss로 sparse_categorical_crossentropy 사용
             metrics=['accuracy'])

# 모델 훈련
model.fit(x_train_norm, y_train, epochs=n_train_epoch)

# 모델 시험
test_loss, test_accuracy = model.evaluate(x_test_norm, y_test, verbose=2)
print("test_loss: {} ".format(test_loss))
print("test_accuracy: {}".format(test_accuracy))


Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_30 (Conv2D)           (None, 26, 26, 32)        896       
_________________________________________________________________
max_pooling2d_25 (MaxPooling (None, 13, 13, 32)        0         
_________________________________________________________________
dropout_20 (Dropout)         (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_31 (Conv2D)           (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_26 (MaxPooling (None, 5, 5, 64)          0         
_________________________________________________________________
dropout_21 (Dropout)         (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_32 (Conv2D)           (None, 3, 3, 128)       

### GAP(Global Average Pooling)의 이점



- FC(Fully Connected)를 사용할 때 위치정보를 잃게 되는데 GAP는 공간정보를 반영하기 때문에 공간적 해석용이
- 파라미터 수를 줄여줄 수 있어 과적합을 피할 수 있음

### 회고

처음에는 코랩을 통해서 실습을 진행했는데 colab 서버에 따라 resize할때 굉장히 시간이 낭비되었었다.
<br>
그래서 다시 아이펠 주피터 클라우드환경에서 데이터 업로드 및 알집 풀기를 다시했어야 했는데 아지트에 선웅님과 형준님께서 올려주신 글덕분에 빠르게 해결할 수 있었다. 
<br>
또한, 데이터도 병휘님께서 양재분들의 데이터를 수집해주셔서 보다 좋은 환경에서 실험을 해볼 수 있었다.
<br>
감사합니다 ㅠㅠ
<br>
첫 실험은 컨볼루션을 2개로 하고, FC로 연결한 구조로부터 시작하여 epoch, filter수, Dense수등의 파라미터를 조절하며 실행을 진행했었다. 이는 train set이 빠르게 accuracy가 증가하였으나 실험을 할때마다 도출된 test accuarcy의 편차가 너무 컸다. 이는 train_set에 대해 과적합이 되고 있는 것을 보여준 것으로 생각된다. 
<br>
그 다음은 컨볼루션을 3개로 하면서 Dropout을 컨볼루션 다음마다 도입하고, FC다음에도 도입해보았다. 0.5이상은 나온것같은데 0.6을 보기에는 힘들었다.
<br>
데이터에 비하여 파라미터 수도 굉장히 많았기 때문에 파라미터를 줄이면서 보다 robust한 모델을 만들려면 어떻게 해야될지 찾아보다가 GAP개념을 발견하였다. FC대신에 GAP를 이용하는 경우에 대한 내용을 살펴보며 도입을 해보았는데 도입 이후부터는 0.6이상의 값들이 잘 나오고 있는 것을 확인할 수 있었다.
<br>
내 생각에는 conv의 filter 수가 커지면 커질수록 parameter수가 늘어나는데 마지막에 GAP가 파라미터 수를 줄여주는 효과 및 공간정보를 함축적으로 요약하며, FC를 연결하여 파라미터수를 늘릴 필요가 없기 때문에 도움이 된것으로 생각된다.